In [0]:


from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, LongType, StringType

date_3 = ['2019-03-31', '2019-03-30', '2019-03-29', '2019-03-28', '2019-03-27', '2019-03-26', '2019-03-25', '2019-03-24', '2019-03-23', '2019-03-22', '2019-03-21', '2019-03-20', '2019-03-19', '2019-03-18', '2019-03-17', '2019-03-16', '2019-03-15', '2019-03-14', '2019-03-13', '2019-03-12', '2019-03-11', '2019-03-10', '2019-03-09', '2019-03-08', '2019-03-07', '2019-03-06', '2019-03-05', '2019-03-04', '2019-03-03', '2019-03-02', '2019-03-01']

date_2 = ['2019-02-28', '2019-02-27', '2019-02-26', '2019-02-25', '2019-02-24', '2019-02-23', '2019-02-22', '2019-02-21', '2019-02-20', '2019-02-19', '2019-02-18', '2019-02-17', '2019-02-16', '2019-02-15', '2019-02-14', '2019-02-13', '2019-02-12', '2019-02-11', '2019-02-10', '2019-02-09', '2019-02-08', '2019-02-07', '2019-02-06', '2019-02-05', '2019-02-04', '2019-02-03', '2019-02-02', '2019-02-01']

rention_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-rt_app.v2/fact/granularity=monthly/"
usage_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=monthly/"
# spark.read.format("parquet").load(rention_path).filter("date between '2019-02-01' and '2019-03-31'").createOrReplaceTempView("test_retention")
spark.read.format("delta").load(usage_path).filter("date between '2019-02-01' and '2019-03-31'").createOrReplaceTempView("test_usage")

_schema =StructType([StructField("daily_date", StringType(), False),StructField("month_date", StringType(), False)])
df_data_2 = [Row(daily_date=r, month_date='2019-02-28') for r in date_2]
df_date_2 = spark.createDataFrame(data=df_data_2, schema=_schema)
df_date_2.createOrReplaceTempView("daily_date_2")

df_data_3 = [Row(daily_date=r, month_date='2019-03-31') for r in date_3]
df_date_3 = spark.createDataFrame(data=df_data_3, schema=_schema)
df_date_3.createOrReplaceTempView("daily_date_3")

spark.sql("select * from daily_date_2 union select * from daily_date_3 order by daily_date desc").createOrReplaceTempView("daily_date")
spark.sql("select * from daily_date dd join test_retention tr on tr.date = dd.month_date and tr.store_id = 10 and tr.gender=0 and tr.app_id=20600004251992 and tr.kpi=301").createOrReplaceTempView("retention_table")

spark.sql("""select rt.*, usage.est_average_active_users  from retention_table rt left join test_usage usage 
on rt.app_id = usage.app_id
and rt.month_date = usage.date
and usage.country_code = 'US'""").createOrReplaceTempView("retention_with_usage")

spark.sql("""select sum(estimate*est_average_active_users) as retention_users, sum(est_average_active_users) 
from retention_with_usage 
where daily_date between '2019-02-24' and '2019-03-02' """).show()



# spark.sql("select * from test_retention limit 1").show(1, False)
# spark.sql("select date, count(*) from test_usage group by date order by date").show(100, False)

# spark.sql("select * from test_retention").show(1, False)
# spark.sql("select distinct kpi from test_retention ").show(100, False)

# spark.sql("select app_id, date, sum(estimate) as estimate from test_retention where app_id = 20600004251992 and date between '2019-02-01' and '2019-08-31'and kpi=301 and store_id=10 group by app_id, date").createOrReplaceTempView("retention_tmp")
# spark.sql("select app_id, date, sum(est_average_active_users) as au from test_usage where app_id = 20600004251992 and date between '2019-02-01' and '2019-08-31' and country_code  in ('US') group by app_id, date").createOrReplaceTempView("usage_tmp")


# spark.sql("""select rt.daily_date, rt.estimate,usage.est_average_active_users from retention_table rt  
# left join test_usage usage
# on rt.app_id = usage.app_id 
# and rt.daily_date = usage.date 
# where rt.app_id = 20600004251992 
# and rt.daily_date between '2019-02-24' and '2019-03-02' 
# and rt.kpi=301 
# and usage.country_code='US' and rt.store_id=10 and rt.gender=0 order by rt.daily_date desc""").createOrReplaceTempView('total_retention_users')

# spark.sql("select * from total_retention_users").show(100, False)


In [0]:

spark.sql("""select rt.*, usage.est_average_active_users, usage.date   from retention_table rt left join test_usage usage 
on rt.app_id = usage.app_id
and rt.month_date = usage.date
and usage.country_code = 'US' order by rt.daily_date desc""").show(1000)

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=daily/date=2019-03

In [0]:


import pandas as pd
from pyspark.sql import functions as F
from datetime import datetime
from applications.db_check_v1.common.db_check_utils import query
from conf.settings import PG_USAGE_HOSTS, PG_USAGE_NAME, PG_USAGE_ACCESS_ID, PG_USAGE_SECRET_KEY, \
    CITUS_USAGE_NAME, CITUS_USAGE_ACCESS_ID, CITUS_USAGE_HOSTS, CITUS_USAGE_SECRET_KEY

PLPROXY_DSN = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_USAGE_NAME,
        user=PG_USAGE_ACCESS_ID,
        host=PG_USAGE_HOSTS[0][0],
        password=PG_USAGE_SECRET_KEY,
        port=PG_USAGE_HOSTS[0][1]
    )
)

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


begin_date = datetime(2019, 2, 1)
end_date = datetime(2019, 3, 31)

DATE_GRANULARITY_MAPPINGLIST = {
    "daily": get_date_list(begin_date, end_date, "D"),
    "weekly": get_date_list(begin_date, end_date, "W-SAT"),
    "monthly": get_date_list(begin_date, end_date, "M")
}


DATE_GRANULARITY_MAPPINGLIST["daily"].reverse()
print DATE_GRANULARITY_MAPPINGLIST["daily"]

In [0]:

rention_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-rt_app.v2/fact/granularity=monthly/"
usage_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=monthly/"
spark.read.format("parquet").load(rention_path).filter("date between '2019-02-01' and '2019-03-31'").createOrReplaceTempView("test_retention")
spark.read.format("parquet").load(usage_path).filter("date between '2019-02-01' and '2019-03-31'").createOrReplaceTempView("test_usage")
spark.sql("select * from test_retention limit 1").show(1, False)
spark.sql("select * from test_usage limit 1").show(1, False)
# spark.sql("select distinct kpi from test_retention ").show(100, False)

# spark.sql("select app_id, date, sum(estimate) as estimate from test_retention where app_id = 20600004251992 and date between '2019-02-01' and '2019-08-31'and kpi=301 and store_id=10 group by app_id, date").createOrReplaceTempView("retention_tmp")
# spark.sql("select app_id, date, sum(est_average_active_users) as au from test_usage where app_id = 20600004251992 and date between '2019-02-01' and '2019-08-31' and country_code  in ('US') group by app_id, date").createOrReplaceTempView("usage_tmp")


spark.sql("""select test_retention.date, test_retention.estimate, test_usage.est_average_active_users from test_retention  
left join test_usage 
on test_retention.app_id = test_usage.app_id 
and test_retention.date = test_usage.date 
where test_retention.app_id = 20600004251992 
and test_retention.date between '2019-02-01' and '2019-03-31' 
and test_retention.kpi=301 
and test_usage.country_code in ('US', 'JP') and test_retention.store_id in (10, 9) and test_retention.gender = 0
and test_retention.age = 0 
and test_retention.device_id=1001 
and test_usage.device_code='android-phone'""").createOrReplaceTempView('total_retention_users')

spark.sql("select * from total_retention_users limit 10").show(10, False)


In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-rt_app.v2/fact/

In [0]:
select 
    sum(au) / sum(pop) as up, 
    sum(atd) / sum(au) as aad, 
    sum(tt) / sum(au) as atu, 
    sum(tv) / sum(au) as afu, 
    sum(tsd) / sum(au) as asd, 
    sum(tt) / sum(tv) as asd_1,
    sum(ib) / sum(pop) as ip,
    sum(tt) as tt,
    sum(tb) / sum(tv) / 1024 / 1024 as abps,
    sum(tb) / sum(au) / 1024 / 1024 as abpu,
    sum(atd) / sum(au) / 7 as atd_w
    --sum(tv) / sum(tsmc) as smc,
    --sum(tt) / sum(ttmc) as tmc
from (
    select 
        date, 
        country_code,
        device_code,
        app_id,
        est_average_session_duration * est_average_active_users / 60 / 1000 as tsd,
        est_population as pop,
        est_average_session_per_user * est_average_active_users as tv,
        est_total_time as tt, 
        est_average_active_days * est_average_active_users as ATD, 
        est_average_active_users AS au,
        est_installs as ib,
        est_average_bytes_per_user * est_average_active_users * est_average_session_per_user as tb
        --est_average_active_users * est_average_session_per_user / est_share_of_category_session as tsmc,
        --est_total_time / 60 / 1000 / est_share_of_category_time as ttmc
        from (
            select * from usage_basic_kpi_fact_v1 
            where 
            app_id in (20600005513979, 1033282601) 
            and date between '2019-10-29' and '2020-09-22' 
            and granularity = 'daily' 
            --and country_code in ('RU', 'US', 'JP', 'IT')
            and country_code in ('US')
            ) tmp1
) tmp2;